In [1]:
import os
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

import dataset
import schedulers

import tensorflow as tf
assert tf.config.list_physical_devices('GPU')

from tensorflow_addons.layers.crf import CRF
from tensorflow_addons.text.crf import crf_log_likelihood

In [2]:

class ModelWithCRFLoss(tf.keras.Model):

    def compute_loss(self, data, training=False):
        x, ys = data
        ys = list(ys.values())
        potentials = []
        crf_losses = []
        preds = self(x, training=training)
        for p, y in zip(preds, ys):
            [_, potential, sequence_length, chain_kernel] = p
            crf_loss = -crf_log_likelihood(potential, y, sequence_length, chain_kernel)[0]
            potentials.append(potential)
            crf_losses.append(crf_loss)
        self.compiled_metrics.update_state(ys, potentials, None)
        return crf_losses

    def log_outputs(self, crf_losses):
        return { # **{f'crf_loss_{i}': x for i, x in enumerate(crf_losses)},
                **{m.name: m.result() for m in self.metrics}}

    def train_step(self, data):
        with tf.GradientTape() as tape:
            crf_losses = self.compute_loss(data, training=True)
            # total_losses[i] = sum(crf_losses[i]) + sum(self.losses[i])

        gradients = tape.gradient(crf_losses, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        return self.log_outputs(crf_losses)

    def test_step(self, data):
        crf_losses = self.compute_loss(data, training=False)
        return self.log_outputs(crf_losses)


BATCH_SIZE = 32

def build_model(UNITS=128):
    LETTERS_SIZE = len(dataset.letters_table)
    NIQQUD_SIZE = len(dataset.niqqud_table)
    DAGESH_SIZE = len(dataset.dagesh_table)
    SIN_SIZE = len(dataset.sin_table)

    inp = keras.Input(batch_shape=(None, None), batch_size=BATCH_SIZE)
    layer = layers.Embedding(LETTERS_SIZE, UNITS, mask_zero=True)(inp)
    layer = layers.Bidirectional(layers.LSTM(UNITS, return_sequences=True), merge_mode='sum')(layer)
    
    outputs = [
        CRF(NIQQUD_SIZE, name='N')(layer),
        CRF(DAGESH_SIZE, name='D')(layer),
        CRF(SIN_SIZE   , name='S')(layer),
    ]
    model = ModelWithCRFLoss(inputs=inp, outputs=outputs)
    model.build((None, None))
    return model

model = build_model()

model.summary()
model.save_weights('./checkpoints/crf_uninit')

Model: "model_with_crf_loss"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 128)    5632        input_1[0][0]                    
__________________________________________________________________________________________________
bidirectional (Bidirectional)   (None, None, 128)    263168      embedding[0][0]                  
__________________________________________________________________________________________________
N (CRF)                         [(None, None), (None 2352        bidirectional[0][0]              
________________________________________________________________________________

In [3]:
def accuracy(y_true, y_pred):
    K = keras.backend
    f = K.floatx()
    # convert dense predictions to labels
    y_pred_labels =  K.cast(K.argmax(y_pred, axis=-1), f)
    
    res = K.cast(K.equal(y_true, y_pred_labels), f)
    return K.sum(res) / K.sum(K.cast(K.not_equal(y_true, 0), f))


def fit(train_validation, lr):
    train, valid = train_validation
    model.compile(metrics="accuracy", loss="ce", optimizer=tf.keras.optimizers.Adam(learning_rate=lr))
    
    y  = {'N': train.niqqud, 'D': train.dagesh, 'S': train.sin }
    vy = {'N': valid.niqqud, 'D': valid.dagesh, 'S': valid.sin }
    return model.fit(train.normalized, y, validation_data=(valid.normalized, vy), batch_size=BATCH_SIZE, epochs=1)


def load_data(source, maxlen=82, validation=0.1):
    filenames = [os.path.join('texts', f) for f in source]
    train, valid = dataset.load_data(filenames, validation, maxlen=maxlen)
    return train, valid

In [4]:
data_mix = load_data(validation=0.1, source=['poetry', 'rabanit', 'pre_modern'])

In [5]:
data_modern = load_data(validation=0.1, source=['modern'])

In [9]:
model.load_weights('./checkpoints/crf_uninit')
history = fit(data_mix, lr=2e-3)
model.save_weights('./checkpoints/crf_mix')

3432/3432 [==============================] - 1119s 326ms/step - N_accuracy: 0.7924 - N_1_accuracy: 0.9041 - N_2_accuracy: 0.9502 - val_N_accuracy: 0.8044 - val_N_1_accuracy: 0.9105 - val_N_2_accuracy: 0.9500


In [14]:
model.load_weights('./checkpoints/crf_mix')
history = fit(data_modern, lr=8e-4)
model.save_weights('./checkpoints/crf_modern')

405/405 [==============================] - 133s 328ms/step - N_accuracy: 0.8181 - N_1_accuracy: 0.9080 - N_2_accuracy: 0.9474 - val_N_accuracy: 0.8145 - val_N_1_accuracy: 0.9024 - val_N_2_accuracy: 0.9429


In [15]:

def print_predictions(data, s):
    batch = data.normalized[s]
    [actual_niqqud, _, _, _], [actual_dagesh, _, _, _], [actual_sin, _, _, _] = model.predict(batch)
    [expected_niqqud, expected_dagesh, expected_sin] = [data.niqqud[s], data.dagesh[s], data.sin[s]]
    actual = dataset.merge(data.text[s], ts=batch, ns=actual_niqqud, ds=actual_dagesh, ss=actual_sin)
    expected = dataset.merge(data.text[s], ts=batch, ns=expected_niqqud, ds=expected_dagesh, ss=expected_sin)
    total = []
    for i, (a, e) in enumerate(zip(actual, expected)):
        print('מצוי: ', a)
        print('רצוי: ', e)
        last = expected_niqqud[i].tolist().index(0)
        res = expected_niqqud[i][:last] == actual_niqqud[i][:last]
        total.extend(res) 
        print(round(np.mean(res), 2), f'({last - sum(res)} out of {last})')
        print()
    print(round(np.mean(total), 3))

model.load_weights('./checkpoints/crf_modern')
print_predictions(data_modern[1], slice(0, 32))

מצוי:  מֶטֶּר. אוֹרֶךְ פְּלָאנְק הוּא הָאוֹרֶךְ הַקְּצֵר בְּיוֹתֵר בַּעַל מַשְׁמָעוּת פִּיזִיקְלִית, וְהוּא בְּ-20 סִדְרִי גּוֹדֶל 
רצוי:  מֶטֶר. אוֹרֶךְ פְּלַאנְק הוּא הָאוֹרֶךְ הַקָּצָר בְּיוֹתֵר בַּעַל מַשְׁמָעוּת פִיזִיקָלִית, וְהוּא בְּ-20 סִדְרֵי גּוֹדֶל 
0.94 (5 out of 78)

מצוי:  בְּאַנְלוּגְיָה לְמִנְעוֹל צִירוּפִים, הִסְתַּמְכּוּת אַךְ וְרַק עַל מִנְגָּנוֹן נְעִילָה סוֹדִי, אֵינָהּ חָכְמָה כִּיוּון 
רצוי:  בְּאָנָלוֹגְיָה לְמַנְעוּל צֵירוּפִים, הִסְתַּמְּכוּת אַךְ וְרַק עַל מַנְגְּנוֹן נְעִילָה סוֹדִי, אֵינָהּ חֲכָמָה כֵּיוָון 
0.85 (12 out of 78)

מצוי:  בִּיקָּשָׁה, שְׁתַּיִּים, גַּם כֵּן קְשׁוּרוֹת.
רצוי:  בִּיקְּשָׁה, שְׁתַּיִים, גַּם כֵּן קְשׁוּרוֹת.
0.96 (1 out of 27)

מצוי:  בֵּינֵיהֶם מִידָּע (כְּלוֹמֶר אַנְרַגְּיָה). כְּשֶׁאָנוּ צוֹפִים לְשַׁמִיִּים הַמֶּרְחָק בֵּין הַפּוֹטוֹנִים שֶׁל קְרִינַת הַרְקֵע 
רצוי:  בֵּינֵיהֶם מֵידָע (כְּלוֹמַר אֵנֶרְגִּיָּה). כְּשֶׁאָנוּ צוֹפִים לַשָּׁמַיִים הַמֶּרְחָק בֵּין הַפוֹטוֹנִים שֶׁל קְרִינַת הָרֶקַע 
0.85 (12 out of 80)

מצוי:  ת